# Advanced spatial joins

## Lecture objectives

1. Explore different spatial relationships (predicates)
2. Practice troubleshooting spatial joins

## Other types of spatial relationships

The `intersects` operator, which we used in the previous lecture, is one of the most common spatial predicates, i.e. the relationship between two geometries. It is excellent if you want to know whether a point is within (or on the boundary) of a polygon. In general, each point will match to one polygon, but you need to be careful if your points are on the boundary and thus intersect with two polygons.

Let's start by loading two of the datasets we used in the last lecture. This is exactly the same code.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import pygris

pantryDf = pd.read_csv('../data/Food_Resources_in_California.csv')
pantryDf = pantryDf[pantryDf.County=='Los Angeles']

# convert to a GeoDataFrame
pantrygdf = gpd.GeoDataFrame(
    pantryDf, geometry=gpd.points_from_xy(pantryDf.Longitude, pantryDf.Latitude, 
                                          crs='EPSG:4326'))
# convert to 3857, so we can match the contextily default for basemaps
pantrygdf.to_crs('EPSG:3857', inplace=True)

# get the census data for LA County
# B19019_001E is median household income
r = requests.get('https://api.census.gov/data/2019/acs/acs5?get=B19019_001E&for=tract:*&in=state:06%20county:037')
censusdata = r.json()
incomeDf = pd.DataFrame(censusdata[1:], columns=censusdata[0])
incomeDf.rename(columns={'B19019_001E':'median_HH_income'}, inplace=True)
incomeDf.median_HH_income = incomeDf.median_HH_income.astype(int)
incomeDf.loc[incomeDf.median_HH_income<0, 'median_HH_income'] = np.nan

# Add the tract boundaries. For this, we'll use the pygris package
tracts = pygris.tracts(state='06',county='037', year=2019)
tracts.set_index(['STATEFP','COUNTYFP','TRACTCE'], inplace=True)
tracts.index.names=['state','county','tract']
# and join the tract boundaries to the census data
incomeDf = tracts[['geometry']].join(incomeDf.set_index(['state','county','tract'])).reset_index()

# create a consolidated GEOID column
incomeDf['GEOID'] = incomeDf.state+incomeDf.county+incomeDf.tract

# reproject to the same crs as pantrygdf
incomeDf = incomeDf.to_crs('EPSG:3857')

Different predicates are defined in the `shapely` [documentation](https://shapely.readthedocs.io/en/stable/manual.html#binary-predicates). (`shapely` is working behind the scenes for many `geopandas` operations.)

In [ ]:
print(incomeDf.sindex.valid_query_predicates)

Let's look at the number of rows that are produced with three different predicates: intersects, contains, and within.

How do you explain the different results below?

In [ ]:
print('There are {} pantries and {} tracts.'.format(len(pantrygdf), len(incomeDf)))

pantries_intersects = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='intersects')
print('Using intersects: {} rows'.format(len(pantries_intersects)))

pantries_contains = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='contains')
print('Using contains: {} rows'.format(len(pantries_contains)))

pantries_within = gpd.sjoin(incomeDf[['GEOID','geometry']], 
                           pantrygdf, how="inner", predicate='within')
print('Using within: {} rows'.format(len(pantries_within)))

Intersects and contains return the same number of rows. If a census tract (a polygon) intersects a pantry (a point geometry), it also contains it.

But a census tract is not within a pantry. A polygon cannot be within a point (it's too big!) 

## Troubleshooting spatial joins

Also note that we have fewer join results than pantries. Why? A good guess would be that some pantries are physically outside the county (the "county" field notwithstanding).

<div class="alert alert-block alert-info">
<strong>Thought exercise:</strong> How would you verify that the missing pantries are outside the county?
</div>

We could map them. But note that the pantries dataset has a County field. So let's do `groupby` and ask for the size of each group. (All of the rows should be Los Angeles county because we filtered when we loaded in the dataset, but maybe we made a mistake?)

In [ ]:
pantrygdf.groupby('County').size()

Hmm. So either all the pantries are within the county, or there is an error in the `County` column. Or our join is going wrong.

Let's map it and take a look. Note that we use the `alpha` parameter to adjust the transparency of both the census tracts and the basemap, making it easier to see the intersection.

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

fig, ax = plt.subplots(figsize=(10,10))
pantrygdf.plot(ax=ax)
incomeDf.plot(ax=ax, alpha=0.5) # 50% opacity
ctx.add_basemap(ax=ax, alpha=0.3) # 30% opacity

Aha. It looks like two of them are just outside the boundary. Probably, the `County` field was misspecified, or the location is imprecise. 

Let's look at the culprits. The `difference` function, which works on a pandas `Index`, is useful here. It's a set-theoretic operation, which looks for the elements that are in one index but not other. [Read about it here.](https://pandas.pydata.org/docs/reference/api/pandas.Index.difference.html)

In [ ]:
pantrygdf.index.difference(pantries_intersects.index)

OK, something went wrong. We should expect two rows (211 in `pantrygdf`, minus 209 in `pantries_intersects`).

In [ ]:
pantries_intersects.head(2)

Aha. The spatial join created a new index, and put the original index in a column called `index_right`. Let's use this column instead. 

In [ ]:
pantrygdf.index.difference(pantries_intersects.index_right)

What are these rows?

In [ ]:
pantrygdf.loc[[390,400]]  

Well, neither of these two cities are in Los Angeles County. (Upland is in San Bernadino, and Los Alamitos is in Orange.) So it's safe to ignore them, and work with the 209 pantries that were spatially joined.

The point here: spatial joins often throw up unexpected results. You'll have to do some detective work to figure out what goes wrong.

## Spatial attributes
We can access useful attributes of the geometry directly from `geopandas`. Areas, lengths, and bounding boxes are three examples.

Note that the units will be based on your projection. You probably don't want to measure distances or areas in degrees! But even a projection in meters can be distorted.

The best projection will vary locally. In the US, the State Plane coordinate systems are usually the best choice for local-level work. Los Angeles is in State Plane [California zone 5, which has an EPSG code of 3497](https://epsg.io/3497). Note the units are in meters. If you want feet, [try EPSG code 6424](https://epsg.io/6424). 

In [ ]:
# in Web Mercator
print(incomeDf.crs)
print(incomeDf.geometry.area.head())

In [ ]:
# in State Plane
print(incomeDf.to_crs('EPSG:3497').geometry.area.head())

For a polygon, length is the perimeter.

In [ ]:
print(incomeDf.geometry.to_crs('EPSG:3497').length.head()) 

The projection is less important for the bounding box, as long as you know which one you are using. Here, we are still in Web Mercator. But lat/lon is often the easiest to work with.

In [ ]:
print(incomeDf.geometry.bounds.head())

In [ ]:
print(incomeDf.to_crs('EPSG:4326').geometry.bounds.head())

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Intersects is the most common spatial predicate, but it's not always what you want.</li>
  <li>Use an an appropriate projection.</li>
  <li>As with any join, inspect your output. If it's not what you expect, computing summary statistics or mapping can help identify the problem.</li>
</ul>
</div>